In [5]:
import pandas as pd
from pycaret.classification import *

In [6]:
# 데이터 로딩
match_data = pd.read_csv('/Users/lhe339/Documents/GitHub/nextorial/data/데이터 분석가_과제_Type A/match_data.csv')
test_data = pd.read_csv('/Users/lhe339/Documents/GitHub/nextorial/data/데이터 분석가_과제_Type A/test_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lhe339/Documents/GitHub/nextorial/data/데이터 분석가_과제_Type A/match_data.csv'

In [ ]:
def calculate_team_features(data):
    return data.groupby(['matchid', 'teamid']).agg({
        'mmr': 'mean',
        'winstreak': 'mean',
        'losestreak': 'mean',
        'recentwinprob': 'mean'
    }).reset_index()

# match_data에 대한 전처리
match_team_features = calculate_team_features(match_data)

# 원래 match_data와 merge하여 각 유저 옆에 팀의 평균 특성값 추가하기
match_data = match_data.merge(match_team_features, on=['matchid', 'teamid'], suffixes=('', '_team_avg'))

In [ ]:
def guild_avg(df):
    # 'guildid'에서 NaN 값을 처리합니다. 이 예에서는 'NoGuild'로 대체합니다.
    df['guildid'].fillna('NoGuild', inplace=True)

    # 티어를 숫자로 변환합니다.
    tier_mapping = {'unranked': 0, 'bronze': 1, 'silver': 2, 'gold': 3, 'platinum': 4, 'diamond': 5, 'master': 6}
    df['tier_numeric'] = df['tier'].map(tier_mapping)

    # 길드별로 데이터를 그룹화하고 그룹별로 평균을 계산합니다.
    guild_avg = df.groupby('guildid')[['mmr', 'winstreak', 'recentwinprob', 'accumatches', 'tier_numeric']].mean()

    # 새로운 컬럼 이름을 생성합니다.
    new_columns = {col: f'{col}_guild_avg' for col in guild_avg.columns}
    guild_avg.rename(columns=new_columns, inplace=True)

    # 원본 데이터셋에 계산된 길드 성능 지표를 병합합니다.
    df = pd.merge(df, guild_avg, on='guildid', how='left')
    
    return df


match_data = guild_avg(match_data)

In [2]:
match_data['guild_membership'] = match_data['guildid'].notna().astype(int)

NameError: name 'match_data' is not defined

In [7]:
# PyCaret 설정
clf1 = setup(data = match_data,
             target = 'matchresult',
             ignore_features = ['createdatekst', 'matchid', 'accountid', 'guildid'],  # 무시할 특성
             ordinal_features = {'tier': ['unranked', 'bronze', 'silver', 'gold', 'platinum', 'diamond', 'master']},  # 순서형 특성
             session_id=123)  # 재현 가능성을 위한 세션 ID 설정

,Description,Value
0,Session id,123
1,Target,matchresult
2,Target type,Binary
3,Original data shape,"(410384, 26)"
4,Transformed data shape,"(410384, 28)"
5,Transformed train set shape,"(287268, 28)"
6,Transformed test set shape,"(123116, 28)"
7,Ignore features,4
8,Ordinal features,1
9,Numeric features,20


In [ ]:
# LightGBM 모델 생성
lightgbm_model = create_model('lightgbm', fold=10)

In [ ]:
# 모델 생성 및 튜닝 (예: Light Gradient Boosting Machine 선택)
lgbm = create_model('lightgbm')
tuned_lgbm = tune_model(lgbm)

In [16]:
pc.evaluate_model(lightgbm_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [17]:
predictions = pc.predict_model(lightgbm_model, data=test_data)

KeyError: "['matchscore', 'isDrop', 'isEscape', 'tier_numeric', 'mmr_guild_avg', 'winstreak_guild_avg', 'recentwinprob_guild_avg', 'accumatches_guild_avg', 'tier_numeric_guild_avg'] not in index"